In [1]:
import pandas as pd
import numpy as np

## 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [2]:
df = pd.read_csv('loans_lenders.csv')
df.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [3]:
# Tramite la funzione lambda ogni row viene convertita da stringa a  lista.

df['lenders'] = df['lenders'].apply(lambda x:x.replace(',','').split())
df.head()

,loan_id,lenders
0,483693,"[muc888, sam4326, camaran3922, lachheb1865, re..."
1,483738,"[muc888, nora3555, williammanashi, barbara5610..."
2,485000,"[muc888, terrystl, richardandsusan8352, sherri..."
3,486087,"[muc888, james5068, rudi5955, daniel9859, don9..."
4,534428,"[muc888, niki3008, teresa9174, mike4896, david..."


In [4]:
# La variabile test indica il numero totale di lenders, anche ripetuti, contenuti nelle liste del dataset.
# Questa variabile verrà utilizzata successivamente per valutare se la funzione explode abbia funzionato 
# correttamente.

test = df['lenders'].apply(lambda x: len(x)).sum()

In [5]:
# Explode ci permette di espandere ogni elemento in ogni lista mantenendo come indice 'loan_id' 
# per ogni membro della lista.

df = df.explode('lenders').reset_index(drop=True)
df.head(10)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
5,483693,karlheinz4543
6,483693,jerrydb
7,483693,paula8951
8,483693,gmct
9,483693,amra9383


In [6]:
# Risultando True l'uguaglianza dimostra come la funzione abbia funzionato alla perfezione.

print(test == len(df))
print(len(df))

True
28293931


## 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [7]:
# Potrebbe essere utile per dopo visualizzare la dimensione del dataframe e le sue colonne. 

df1 = pd.read_csv('loans.csv')
print('Shape:', df1.shape)
print('')
print('Columns:', df1.columns)

Shape: (1419607, 31)

Columns: Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')


In [8]:
# La variabile null_sum indica la somma di valori Na tra le due variabili che verranno utilizzate
# in seguito.

null_sum = df1['disburse_time'].isnull().sum() + df1['planned_expiration_time'].isnull().sum()

In [9]:
# Inters indica invece l'intersezione dei Na tra le due variabili, ovvero i Na sia per l'una che per 
# l'altra variabile.

inters = df1['disburse_time'][(df1['disburse_time'].isnull()==True)&(df1['planned_expiration_time'].isnull()==True)]

In [10]:
# Conversione delle variabili nel formato pandas datetime.

df1['planned_expiration_time'] = pd.to_datetime(df1['planned_expiration_time'])
df1['disburse_time'] = pd.to_datetime(df1['disburse_time'])

In [11]:
df1['duration'] = (df1['planned_expiration_time'] - df1['disburse_time']).dt.days

In [12]:
# La colonna appena creata permette di visualizzare la differenza in giorni tra le due variabili. 

df1['duration'].head()

0    53.0
1    96.0
2    37.0
3    34.0
4    57.0
Name: duration, dtype: float64

In [13]:
# Come previsto gli Na sono rimasti invariati. 

(null_sum - len(inters)) == df1['duration'].isnull().sum()

True

## 3.Find the lenders that have funded at least twice.

In [14]:
# Con il groupby è possibile raggruppare per lenders in modo da vedere quante volte compare in ogni 
# prestito all'interno del primo dataframe. Successivamente si scelgono solamente coloro che sono presenti 
# almeno in due prestiti differenti. 

group = df.groupby('lenders').size()
group = group[group >= 2].reset_index(name='Count')
group.head()

,lenders,Count
0,000,40
1,00000,39
2,0002,70
3,0101craign0101,71
4,0132575,4


In [15]:
group['lenders']

0                    000
1                  00000
2                   0002
3         0101craign0101
4                0132575
               ...      
901806           zzinnia
901807        zzivnk7094
901808          zzmcfate
901809        zzrvmf8538
901810            zzzsai
Name: lenders, Length: 901811, dtype: object

In [16]:
print('I lenders presenti in più di un prestito sono {}, rappresentanti il {}% dei lenders totali.'
      .format(len(group), round(len(group)/df.lenders.nunique(), 2)*100))

I lenders presenti in più di un prestito sono 901811, rappresentanti il 65.0% dei lenders totali.


## 4. For each country, compute how many loans have involved that country as borrowers.

In [17]:
# Numero di prestiti effettuati per ogni stato.

loan_country = df1.groupby('country_name')['loan_id'].size().reset_index(name = 'loans_count')
loan_country.head()

,country_name,loans_count
0,Afghanistan,2337
1,Albania,3075
2,Armenia,13952
3,Azerbaijan,10172
4,Belize,218


## 5. For each country, compute the overall amount of money borrowed.

In [18]:
# Totale finanziato per ogni stato

total_country = df1.groupby('country_name')['loan_amount'].sum().reset_index(name = 'total_borrowed_amount')
total_country.head()

,country_name,total_borrowed_amount
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0


## 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [19]:
# Come il punto precedente ma dividendo ogni country con il totale di 'loan_amount'

perc_country = df1.groupby('country_name')['loan_amount'].sum() / df1['funded_amount'].sum()*100
perc_country = perc_country.reset_index(name = '%_over_total')
perc_country.head()

,country_name,%_over_total
0,Afghanistan,0.174126
1,Albania,0.381119
2,Armenia,2.030683
3,Azerbaijan,1.308160
4,Belize,0.013288


## 7. Like the three previous points, but split for each year (with respect to disburse time).

In [20]:
# Viene creata una nuova variabile indentificante l'anno

df1['years'] = df1['disburse_time'].dt.year

In [21]:
print('Years Na:', df1.years.isnull().sum())

Years Na: 2813


In [22]:
# Conteggio dei prestiti in base allo stato e all'anno.

df1.groupby(['country_name','years'])['loan_id'].size().reset_index(name='count')

,country_name,years,count
0,Afghanistan,2007.0,408
1,Afghanistan,2008.0,370
2,Afghanistan,2009.0,678
3,Afghanistan,2010.0,632
4,Afghanistan,2011.0,247
...,...,...,...
743,Zimbabwe,2013.0,426
744,Zimbabwe,2014.0,2078
745,Zimbabwe,2015.0,600
746,Zimbabwe,2016.0,808


In [23]:
# Importo totale per anno e stato.

df1.groupby(['country_name','years'])['loan_amount'].sum().reset_index(name='total_loan_amount')

,country_name,years,total_loan_amount
0,Afghanistan,2007.0,194975.0
1,Afghanistan,2008.0,365375.0
2,Afghanistan,2009.0,585125.0
3,Afghanistan,2010.0,563350.0
4,Afghanistan,2011.0,245125.0
...,...,...,...
743,Zimbabwe,2013.0,678525.0
744,Zimbabwe,2014.0,1311575.0
745,Zimbabwe,2015.0,723625.0
746,Zimbabwe,2016.0,788600.0


In [24]:
# Rapporto tra amount e funded per anno e stato.

(df1.groupby(['country_name','years'])['loan_amount'].sum() / df1['funded_amount'].sum()*100).reset_index(
 name='percent_over_total')

,country_name,years,percent_over_total
0,Afghanistan,2007.0,0.017252
1,Afghanistan,2008.0,0.032329
2,Afghanistan,2009.0,0.051772
3,Afghanistan,2010.0,0.049846
4,Afghanistan,2011.0,0.021689
...,...,...,...
743,Zimbabwe,2013.0,0.060037
744,Zimbabwe,2014.0,0.116050
745,Zimbabwe,2015.0,0.064027
746,Zimbabwe,2016.0,0.069776


## 8. For each lender, compute the overall amount of money lent.  For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [25]:
# Merge su loan_id in modo da avere anche il nome di ogni lender.

df2 = df1[['loan_id','loan_amount','funded_amount']].merge(df, on='loan_id')
df2.head()

,loan_id,loan_amount,funded_amount,lenders
0,657307,125.0,125.0,spencer5657
1,657307,125.0,125.0,matthew8640
2,657307,125.0,125.0,larry71496105
3,657259,400.0,400.0,ltr
4,657259,400.0,400.0,andrew5306


In [26]:
# numero di lenders per ogni prestito.

id_group = df2.groupby('loan_id').size()
id_group.head()

loan_id
84    3
85    2
86    3
88    3
89    4
dtype: int64

In [27]:
# Con il groupby faccio in modo che i loan_id della tabella siano ordinati come quelli della numerosità.

funded_group = df2.groupby(['loan_id','lenders'])['funded_amount'].sum()
funded_group.head()

loan_id  lenders
84       brooke     500.0
         michael    500.0
         ward       500.0
85       michael    500.0
         patrick    500.0
Name: funded_amount, dtype: float64

In [28]:
# divisione tra somma prestata e numerosità di ciascun gruppo

funded_group = funded_group/id_group

In [29]:
# Somma di denaro conferita da ogni singolo lender.

funded_group = funded_group.reset_index(name = 'funded_amount_each')
funded_group.head()

,loan_id,lenders,funded_amount_each
0,84,brooke,166.666667
1,84,michael,166.666667
2,84,ward,166.666667
3,85,michael,250.000000
4,85,patrick,250.000000


In [30]:
# Raggruppamento per lender e somma totale.

funded_group.groupby('lenders')['funded_amount_each'].sum().reset_index(name = 'total_lent')

,lenders,total_lent
0,000,1703.868411
1,00000,1379.750248
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1383794,zzmcfate,63381.546705
1383795,zzpaghetti9994,51.020408
1383796,zzrvmf8538,513.213719
1383797,zzzsai,267.667370


## 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [31]:
id_group = id_group.reset_index(name='count')
id_group.head()

,loan_id,count
0,84,3
1,85,2
2,86,3
3,88,3
4,89,4


In [32]:
# df2 = df2[df2['funded_amount']-df2.['loan_amount'] != 0] # Per eliminare le differenze uguali a zero.

first = (df2.groupby(['loan_id','lenders'])['funded_amount','loan_amount'].sum()).reset_index()
second = first.merge(id_group,on='loan_id')
second.head()

,loan_id,lenders,funded_amount,loan_amount,count
0,84,brooke,500.0,500.0,3
1,84,michael,500.0,500.0,3
2,84,ward,500.0,500.0,3
3,85,michael,500.0,500.0,2
4,85,patrick,500.0,500.0,2


In [33]:
second['funded_amount'] = second['funded_amount']/second['count']
second['loan_amount'] = second['loan_amount']/second['count']
second.head()

,loan_id,lenders,funded_amount,loan_amount,count
0,84,brooke,166.666667,166.666667,3
1,84,michael,166.666667,166.666667,3
2,84,ward,166.666667,166.666667,3
3,85,michael,250.000000,250.000000,2
4,85,patrick,250.000000,250.000000,2


In [34]:
# Caricamento del dataset relativo ai lenders.
# Na drop pr le variabili che verranno utilizzate e merge con il df precedente

df_lenders= pd.read_csv('lenders.csv')
df_lenders.dropna(subset=['country_code',],inplace=True)
df_lenders.dropna(subset=['permanent_name'],inplace=True)
df_lenders.rename(columns={'permanent_name':'lenders'}, inplace=True)
# Merge con il primo dataframe per avere loan_id,lenders e country_code (relativo a lenders).

lendersWCountry = df.merge(df_lenders[['lenders', 'country_code']], on='lenders')
lendersWCountry.dropna(inplace=True)
lendersWCountry.head()

,loan_id,lenders,country_code
0,483693,muc888,US
1,483738,muc888,US
2,485000,muc888,US
3,486087,muc888,US
4,534428,muc888,US


In [35]:
# Merge finale con due variabili come superchiave per indicare la nazionalità di ogni lender.

finale = second.merge(lendersWCountry,on=['loan_id','lenders'])
finale

,loan_id,lenders,funded_amount,loan_amount,count,country_code
0,84,brooke,166.666667,166.666667,3,US
1,84,michael,166.666667,166.666667,3,US
2,84,ward,166.666667,166.666667,3,US
3,85,michael,250.000000,250.000000,2,US
4,85,patrick,250.000000,250.000000,2,US
...,...,...,...,...,...,...
23052530,1444017,deveda8228,50.000000,125.000000,1,US
23052531,1444047,lawrence1598,50.000000,50.000000,1,HK
23052532,1444051,lauren1424,50.000000,225.000000,1,US
23052533,1444053,dario6858,50.000000,725.000000,1,GB


In [36]:
finale['difference'] = finale['loan_amount'] - finale['funded_amount']

In [37]:
finale = finale.groupby('country_code')['loan_amount','funded_amount','difference'].sum().reset_index()
finale

,country_code,loan_amount,funded_amount,difference
0,AD,5.124011e+03,5.060428e+03,63.583333
1,AE,1.963471e+06,1.914735e+06,48736.111556
2,AF,1.119684e+05,1.081489e+05,3819.582746
3,AG,6.182085e+02,6.182085e+02,0.000000
4,AI,3.350203e+02,3.350203e+02,0.000000
...,...,...,...,...
220,YE,2.061570e+04,2.056788e+04,47.818396
221,YT,5.970588e+01,5.970588e+01,0.000000
222,ZA,5.612558e+05,5.511354e+05,10120.422294
223,ZM,3.576965e+04,2.971218e+04,6057.466353


# 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [38]:
# La variabile population è presente nel dataset country_stats.csv

df6 = pd.read_csv('C:/Users/giaco/Desktop/Dataset Esame CS/country_stats.csv')
df6.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


In [39]:
# Viene eseguito un merge per utlizizzare i valori trovati nel punto precedente

df7 = df6[['country_code','population']].merge(finale,on='country_code')
df7.head()

,country_code,population,loan_amount,funded_amount,difference
0,IN,1339180127,1.051133e+06,1.032385e+06,18748.435665
1,NG,190886311,3.630694e+04,3.560670e+04,700.235426
2,MX,129163276,1.467074e+06,1.421844e+06,45229.725459
3,PK,197015955,9.978677e+04,9.870512e+04,1081.645655
4,BD,164669751,2.444878e+04,2.405690e+04,391.884560


In [40]:
df7['ratio']= df7.difference/df7.population
df7.head()

,country_code,population,loan_amount,funded_amount,difference,ratio
0,IN,1339180127,1.051133e+06,1.032385e+06,18748.435665,0.000014
1,NG,190886311,3.630694e+04,3.560670e+04,700.235426,0.000004
2,MX,129163276,1.467074e+06,1.421844e+06,45229.725459,0.000350
3,PK,197015955,9.978677e+04,9.870512e+04,1081.645655,0.000005
4,BD,164669751,2.444878e+04,2.405690e+04,391.884560,0.000002


In [41]:
# La nazione con il ratio maggiore

df7[df7.ratio == df7.ratio.max()]

,country_code,population,loan_amount,funded_amount,difference,ratio
156,NO,5305383,1.966635e+07,1.873409e+07,932265.20019,0.175721


## 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [42]:
df6[['country_code','population','population_below_poverty_line']].head()

,country_code,population,population_below_poverty_line
0,IN,1339180127,21.9
1,NG,190886311,70.0
2,MX,129163276,46.2
3,PK,197015955,29.5
4,BD,164669751,31.5


In [43]:
df6['population_below_poverty_line'] = df6.population_below_poverty_line/100
df6['population_not_below_poverty_line'] = 1 - df6.population_below_poverty_line

In [44]:
df8 = df6[['country_code','population','population_not_below_poverty_line']].merge(finale[['country_code','difference']],on='country_code')

In [45]:
df8['ratio'] = df8['difference']/(df8['population']*df8['population_not_below_poverty_line'])
df8.head()

,country_code,population,population_not_below_poverty_line,difference,ratio
0,IN,1339180127,0.781,18748.435665,0.000018
1,NG,190886311,0.300,700.235426,0.000012
2,MX,129163276,0.538,45229.725459,0.000651
3,PK,197015955,0.705,1081.645655,0.000008
4,BD,164669751,0.685,391.884560,0.000003


In [46]:
df8[df8.ratio == df8.ratio.max()]

,country_code,population,population_not_below_poverty_line,difference,ratio
6,US,324459463,0.849,2.165696e+07,0.078619


## 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [47]:
# Viene utilizzato il dataframe loans.csv.
# Vengono rimossi gli NA per le variabili seguenti ricordandoci che erano gia state convertite nel
# formato datetime nel punto 2.

df1.dropna(subset=['planned_expiration_time'],inplace=True)
df1.dropna(subset=['disburse_time'],inplace=True)

In [48]:
# Vengono create delle nuove variabili: le prime due rappresentato rispettivamente la data di inizio e la
# data di completamento del prestito, le successive due rappresentano i giorni di competenza del primo anno
# e la quarta i giorni di competenza dell'ultimo. L'ultima variabile indica la differenza in termini di
# anno (rispettivamente all'ano di stipulazione e l'anno di cessazione del prestito)

df1['year_start']= df1['disburse_time'].dt.year
df1['year_end']=df1['planned_expiration_time'].dt.year
df1['days_first_year']= 365 - df1['disburse_time'].dt.dayofyear
df1['days_last_year']=df1['planned_expiration_time'].dt.dayofyear
df1['difference_years']=df1['year_end']-df1['year_start']

In [49]:
# Il ciclo sottostante serve a creae ulteriori variabili settate con valore 0 per ogni anno.
# Queste colonne andranno poi riempite con la quota di competenza del rispettivo anno.

bound = range(df1.year_start.min(),df1.year_end.max()+1)
for i in bound:
    df1[str(i)] = 0

In [50]:
# Variabili utilizzate

atribute = ['loan_amount', '2011', '2012', '2013', '2014','2015', '2016', 
            '2017', '2018', 'difference_years', 'days_first_year',
            'days_last_year', 'year_start', 'year_end']

# Con il seguente comando rimuoviamo tutte le osservazioni che presentano una data di disburse_time superiore
# a planned_expiration_time. 

df1 = df1[atribute][(df1.difference_years != -1)]
df1.head()

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018,difference_years,days_first_year,days_last_year,year_start,year_end
0,125.0,0,0,0,0,0,0,0,0,1,9,45,2013,2014
1,400.0,0,0,0,0,0,0,0,0,1,11,85,2013,2014
2,400.0,0,0,0,0,0,0,0,0,0,356,46,2014,2014
3,625.0,0,0,0,0,0,0,0,0,0,348,52,2014,2014
4,425.0,0,0,0,0,0,0,0,0,1,14,44,2013,2014


In [51]:
# Questa funzione ci permette di calcolare la quota di prestito appartenende ad ogni anno compreso tra
# disburse_time e planned_expiration_time

def loan(x):
    if x['difference_years'] == 0: # se disburse e planned hanno stipulazione e scadenza nello stesso anno.
        x[str(int(x['year_start']))]= x['loan_amount'] # l'intero ammontare di loan_amount viene inserito 
                                                       # nella colonna in corrispondenza del medesimo anno.
    elif x['difference_years']== 1:
        x[str(int(x['year_start']))]= (x['days_first_year']*x['loan_amount'])/((x['days_first_year'])+ x['days_last_year']+ ((x['difference_years']-1)*365))
        x[str(int(x['year_end']))]= x['loan_amount'] - x[str(int(x['year_start']))]
        
    elif x['difference_years'] > 1:
        x[str(int(x['year_start']))]= x['days_first_year']*x['loan_amount'] / (x['days_first_year'] + x['days_last_year'] + ((x['difference_years']-1)*365))
        x[str(int(x['year_end']))]=(x['days_last_year']*x['loan_amount'])/((x['days_first_year'])+ x['days_last_year']+ ((x['difference_years']-1)*365))
        
        for y in range(1,int(x['difference_years'])): # ciclo di riempimento per gli anni compresi
            x[str(int(x['year_start']+y))] = (365*x['loan_amount'])/((x['days_first_year'])+ x['days_last_year']+ ((x['difference_years']-1)*365))
    return x

In [52]:
df1 = df1.apply(loan,axis=1)
df1.head()

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018,difference_years,days_first_year,days_last_year,year_start,year_end
0,125.0,0.0,0.0,20.833333,104.166667,0.0,0.0,0.0,0.0,1.0,9.0,45.0,2013.0,2014.0
1,400.0,0.0,0.0,45.833333,354.166667,0.0,0.0,0.0,0.0,1.0,11.0,85.0,2013.0,2014.0
2,400.0,0.0,0.0,0.000000,400.000000,0.0,0.0,0.0,0.0,0.0,356.0,46.0,2014.0,2014.0
3,625.0,0.0,0.0,0.000000,625.000000,0.0,0.0,0.0,0.0,0.0,348.0,52.0,2014.0,2014.0
4,425.0,0.0,0.0,102.586207,322.413793,0.0,0.0,0.0,0.0,1.0,14.0,44.0,2013.0,2014.0


In [53]:
# La funzione sembra operare corretamente con tutte le differenze tra gli anni

df1[df1.difference_years > 1]

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018,difference_years,days_first_year,days_last_year,year_start,year_end
33780,650.0,3.545455,143.787879,143.787879,143.787879,143.787879,71.303030,0.0,0.0,5.0,9.0,181.0,2011.0,2016.0
33781,525.0,0.000000,82.237277,126.652346,126.652346,126.652346,62.805684,0.0,0.0,4.0,237.0,181.0,2012.0,2016.0
33859,600.0,0.000000,50.000000,157.327586,157.327586,157.327586,78.017241,0.0,0.0,4.0,116.0,181.0,2012.0,2016.0
33923,250.0,0.000000,31.656400,62.457221,62.457221,62.457221,30.971937,0.0,0.0,4.0,185.0,181.0,2012.0,2016.0
34072,5000.0,30.284676,1105.390672,1105.390672,1105.390672,1105.390672,548.152635,0.0,0.0,5.0,10.0,181.0,2011.0,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376686,100.0,0.000000,6.520147,26.739927,26.739927,26.739927,13.260073,0.0,0.0,4.0,89.0,181.0,2012.0,2016.0
1397645,100.0,0.000000,5.830258,26.937269,26.937269,26.937269,13.357934,0.0,0.0,4.0,79.0,181.0,2012.0,2016.0
1397865,50.0,0.000000,6.331280,12.491444,12.491444,12.491444,6.194387,0.0,0.0,4.0,185.0,181.0,2012.0,2016.0
1398082,525.0,0.000000,82.237277,126.652346,126.652346,126.652346,62.805684,0.0,0.0,4.0,237.0,181.0,2012.0,2016.0


In [54]:
# Vengono selezionate solo le colonne relative agli anni e computato il totale su di esse.

just_years = [(lambda x: str(x))(x) for x in list(bound)] 
df1[just_years].apply(lambda x: int(x.sum())).reset_index().rename(columns={'index':'year',0:'total'})

,year,total
0,2011,629911
1,2012,113122955
2,2013,125089016
3,2014,153151119
4,2015,159759960
5,2016,158748741
6,2017,172100797
7,2018,5705070
